# Dinozor NLP 

Our goal is to to demonstrate an old NLP task with old NLP methodologies, to understand what the future methods are trying to do better. For this goal I found a Turkish SMS Spam detection dataset from [Onur Karasoy et al.](https://github.com/onrkrsy/TurkishSMS-Collection)

In [ ]:
import pandas as pd
from pathlib import Path

dataset_path = Path("TurkishSMS-Collection/TurkishSMSCollection.csv")
df = pd.read_csv(dataset_path, sep=';')

This is how the data looks like

In [ ]:
df

The classes are balanced which is nice

In [ ]:
df.Group.value_counts()

I am turning classes into 0 and 1 for convenience

In [ ]:
df["Group"] = df["Group"].replace(2, 0)

## Good Old Feature Engineering

Please Look at some samples and try to come up with some features that distinguish between spam and ham in this dataset. Than a bunch of classifiers will take those as inputs to generate scores. It is not about the results but the process of applying ancient ML methodologies for real life NLP problems.

In [ ]:
pd.set_option('display.max_colwidth', None)

Take a look at random samples from each classes and try to come up with features that differanciates onw from the other

In [ ]:
df[df["Group"] == 1].sample(5)

In [ ]:
df[df["Group"] == 0].sample(5)

### Exercise 1

Below here, engineer some features and append them to the original dataframe like:

```python
def get_my_feature(text):
    # calculate your galaxy brain feature
    text = text.do_stuff()
    return text

df["my_feature"] = df["Messages"].apply(get_my_feature)
```
or in any way you like

take a look at your newly engineered features

In [ ]:
df

### Lazy classifier

Grinding different models to hit a higher score is automatable. What we are doing here is only meaningful during benchmarking, productionizing our solutions would bring up different concerns

In [ ]:
def train_test_split(df: pd.DataFrame, target: str, ratio: float=0.3): # i know i didn't need to write this
    X = df.drop(target, axis=1)
    Y = df[[target]]
    split = round(len(df)*ratio)
    X_test = X.iloc[:split]
    X_train = X.iloc[split:]
    y_test = Y.iloc[:split]
    y_train = Y.iloc[split:]
    return X_train, X_test, y_train, y_test

I am using a library called lazy predict which is basically goes around and tries every sklearn classifier on your data, so that we can ignore model selection and hyperparameter tuning and just focus on the data

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import precision_score

## write here your features along with the Group column
features = df[["<put your features here alongside the Group column>", "Group"]]

X_train, X_test, y_train, y_test = train_test_split(features, target="Group", ratio=.3)

clf = LazyClassifier(verbose=0, ignore_warnings=False, predictions=True, random_state=42, classifiers="all", custom_metric=precision_score)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

Let's make up a business rule and say that our spam filtering system must avoid flagging our loved ones' SMSs as spams.

So let's compare classifiers by precision

In [ ]:
models.sort_values("precision_score", ascending=False)

___

Here is a simple error analysis view so you can see what sort of examples are predicted wrong and develop features based on your hypotheses

In [ ]:
predictions["Group"] = y_test
highest_precision_classifier = models.sort_values("precision_score", ascending=False).index[0]

# the examples that were not spams but the best classifier decided otherwise
indices = predictions[(predictions["Group"] == 0) & (predictions[highest_precision_classifier] == 1)].index
df.iloc[indices]

What do you think could be improved?

## Term document matrix

Since the dawn of time, the goal of NLP research is to somehow represent language units with numbers. Because only then, we can make data science with them

![xkcd](https://imgs.xkcd.com/comics/assigning_numbers.png)

One of the older ways to represent words (or tokens) and documents was to create a term-document matrix. We can assume in such matrix the rows are words and columns are documents, and the cells are a function of those two. The most basic function to use might be the frequency of that word in a document. With that we are representing each document with a vacabulary-size dimentional sparse vector. It is also called a co-occurance matrix. Words that co-occur are represented by vectors that are closer.  
For example the words "volkan" and "konak" might occur together more often than "volkan" and "şemsiye"; Therefore distance("volkan", "konak") < distance("volkan", "şemsiye)

The assumption we are making is: **The meaning of documents are a function of the words they contain**  
Let's build that!

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample texts
texts = df["Message"].values

# Create term-document matrix
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(texts)

# Convert to DataFrame for better visualization
td_matrix = pd.DataFrame(matrix.toarray(), 
                 columns=vectorizer.get_feature_names_out(),
                 index=[f'Doc{i+1}' for i in range(len(texts))])

print("Term-document matrix:")
td_matrix

Very simple. You see the rows are documents and the columns are 'words'. We have word representations based on which documents they occur in (is this language modelling??) and we have document representations based on how many of each word they contain.

let's see the most frequent words

In [ ]:
td_matrix.sum().sort_values(ascending=False).head(10)

We can infer our mostly co-occured words via vector similarity

In [ ]:
def get_most_cooccurances(word, matrix_df, top_k=10):
    if word not in matrix_df.columns:
        raise Exception(f"{word} does not exist in the vocabulary")
    vec = td_matrix[word].values
    similarities = vec.dot(matrix.toarray())
    top_k_indices = (-similarities).argsort()[:top_k]
    return [matrix_df.iloc[:, i].name for i in top_k_indices]

go ahead and discover what words co occur mostly, you can filter by spamness to infer how cooccurance differs between two classes

In [ ]:
get_most_cooccurances("cumalar", td_matrix)

It sort of makes sense, but why is it so ugly?  
The words are extracted naively. There are different vectors for "düşünmek", "düşünüyorum", "düşündüler" etc.
Our assumption was that document meaning is a function of its words. We can also assume these words would contribute to similar meanings in a document, so treating them as seperate creates noise.  
Also, words like "ve", "veya", "şöyle", "böyle" should contribute very little to the meaning. Getting rid of those should also remove some of the noise in the matrix

### Exercise 2 
go ahead and write preprocessing step to mitigate the problems stated above, you might remember terms like stop words, stemming and so on. Then, use your new and beautiful term occurances as features for the classifiers above and see how well it performs compared to your initial feature engineering.

# NLP Tasks

## Token Classification

In [ ]:
from tqdm import tqdm
tqdm.pandas()

a tokenizer will be useful

In [ ]:
from transformers import AutoTokenizer
model_name = "dbmdz/distilbert-base-turkish-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

### NER

Token classification is about classifying the parts (words, subwords...) of a text.

Most known application is Named Entity Recognition:

- [ "My", "name", "is", "Ahmet", "." ]
- [ "O", "O", "O", "PERSON", "O" ]  

Named entity recognition finds the special entities in a text, such as "person", "location", "date".

It is a type of token classification, classes being, for example, "O", "PERSON", "LOC", "DATE".

#### How does the ner data look like?

[turkish-nlp-suite/turkish-wikiNER](https://huggingface.co/datasets/turkish-nlp-suite/turkish-wikiNER)  
[aynısının github linki](https://github.com/turkish-nlp-suite/Turkish-Wiki-NER-Dataset/)


I am reading the same data as pandas dataframe and huggingface Datasets to understand what Datasets has to offer and how do they differ

In [ ]:
# Loading dataset via pandas
import pandas as pd

splits = {'train': 'dataset/train.json', 'validation': 'dataset/valid.json', 'test': 'dataset/test.json'}
df = pd.read_json("hf://datasets/turkish-nlp-suite/turkish-wikiNER/" + splits["train"], lines=True)

These are the classes represented in the dataset

In [ ]:
label_list = ['O',
'B-CARDINAL',
'I-CARDINAL',
'B-DATE',
'I-DATE',
'B-EVENT',
'I-EVENT',
'B-FAC',
'I-FAC',
'B-GPE',
'I-GPE',
'B-LANGUAGE',
'I-LANGUAGE',
'B-LAW',
'I-LAW',
'B-LOC',
'I-LOC',
'B-MONEY',
'I-MONEY',
'B-NORP',
'I-NORP',
'B-ORDINAL',
'I-ORDINAL',
'B-ORG',
'I-ORG',
'B-PERCENT',
'I-PERCENT',
'B-PERSON',
'I-PERSON',
'B-PRODUCT',
'I-PRODUCT',
'B-QUANTITY',
'I-QUANTITY',
'B-TIME',
'I-TIME',
'B-TITLE',
'I-TITLE',
'B-WORK_OF_ART',
'I-WORK_OF_ART']

Let's take a look at what we are dealing with

In [ ]:
df

Here we see the labels are given for each word. But most modern approaches don't use word tokenization. We also will be using a model with subword tokenization. Subword tokenization is very beneficial with morphologically rich languages like Turkish.

In the function below we are aligning the labels with the actual tokens that our model will use.  
Feel free to disect it

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. For more info check the [original notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=DIba90p4rvU_)

In [ ]:
# How would the code change if we just assume we only want to label all tokens?

label_all_tokens=True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 truncation=True,
                                 is_split_into_words=True)

    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
         # Special tokens have a word id that is None. We set the label to -100 so they are automatically
        # ignored in the loss function.
        if word_idx is None:
            label_ids.append(-100)
        # We set the label for the first token of each word.
        elif word_idx != previous_word_idx:
            label_ids.append(label_list.index(examples["tags"][word_idx]))
        # For the other tokens in a word, we set the label to either the current label or -100, depending on
        # the label_all_tokens flag.
        else:
            label_ids.append(label_list.index(examples["tags"][word_idx]) if label_all_tokens else -100)
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = label_ids
    #import pdb; pdb.set_trace()
    return tokenized_inputs

In [ ]:
tmp_df = df.progress_apply(tokenize_and_align_labels, axis=1)

In [ ]:
tokenized_df = pd.DataFrame(tmp_df.tolist()) # burayı başka bi şekilde yap

This is how the tokenized labels look like

In [ ]:
# we can also add the decoded input_ids to peep into the tokenization of the actual text
tokenized_df

#### Finetuning NER

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

In [ ]:
from transformers import DataCollatorForTokenClassification
# Data collator that will dynamically pad the inputs received, as well as the labels.
data_collator = DataCollatorForTokenClassification(tokenizer)

It is a very convenient abstraction to use datasets library with transformers feel free to check how it differs from pandas df

In [ ]:
import datasets
split = round(len(tokenized_df)*0.3)
print(split)

dataset = datasets.DatasetDict(
    {
        "train": datasets.Dataset.from_pandas(tokenized_df[split:]),
        "test": datasets.Dataset.from_pandas(tokenized_df[:split]),
    }
)

In [ ]:
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np
from seqeval.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(y_true=true_labels, y_pred=true_predictions),
        "recall": recall_score(y_true=true_labels, y_pred=true_predictions),
        "f1": f1_score(y_true=true_labels, y_pred=true_predictions),
        "accuracy": accuracy_score(y_true=true_labels, y_pred=true_predictions)
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

I hope the next cell does not start your pc fan immediately

In [ ]:
trainer.train()

Let's see how we did on the test set

In [ ]:
trainer.evaluate()

In [ ]:
def compute_test_results():
    predictions, labels, _ = trainer.predict(dataset["test"])
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return true_predictions, true_labels


In [ ]:
pred, label = compute_test_results()

We can see example-wise accuracy scores

In [ ]:
for p, l in zip(pred, label):
    a = [pp==ll for pp,ll in zip(p,l)]
    print(sum(a)/len(a))

#### NER Inference

Feel free to play with your examples to see what the model is good and bad at

In [ ]:
example_sentence = "Inzva'nın Taksim binasını Yağız hiç görmemiş."

In [ ]:
inputs = tokenizer(example_sentence, return_tensors="pt", add_special_tokens=True)
inputs["input_ids"] = inputs["input_ids"].to(device=model.device)
inputs["attention_mask"] = inputs["attention_mask"].to(device=model.device)


In [ ]:
outputs = model(**inputs)

In [ ]:
predicted_classes = outputs['logits'].argmax(axis=2).cpu().numpy()[0]

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(ids=inputs["input_ids"].cpu().numpy()[0], skip_special_tokens=False)

In [ ]:
for i, p in enumerate(predicted_classes):
    if tokens[i] in [tokenizer.sep_token, tokenizer.cls_token]:
        continue
    print(f"{tokens[i]} ----> {label_list[p]}")

### Extractive QA

Extractive QA can also be formulated as a token classification problem. Here extractive means that the answers is a span inside the given context. So we can train a model to predict for each token to find which token is the start token and which token is the end token.

This is what the SQuAD data format looks like which is quite a common standard dataset and format for QA literature (a bit outdated imo)

In [ ]:
example_qa = {
                "data": [
                    {
                        "title": "Example",
                        "paragraphs": [
                            {
                                "context": "The quick brown fox jumps over the lazy dog.",
                                "qas": [
                                    {
                                        "question": "What does the fox jump over?",
                                        "id": "q1",
                                        "answers": [
                                            {
                                                "text": "the lazy dog",
                                                "answer_start": 32
                                            }
                                        ]
                                    }
                                ]
                            }
                        ]
                    }
                ],
                "version": "2.0"
            }

We will be demonstrating the Extractive QA Task with a translated SQuAD dataset. From our friends at Boun-tabilab
[boun-tabi/squad_tr](https://huggingface.co/datasets/boun-tabi/squad_tr)

In [ ]:
import gzip
import json

with gzip.open("SQuAD-TR/data/squad-tr-train-v1.0.0.json.gz", "r") as f:
   qa_data = json.loads(f.read().decode('utf-8'))

This time we are directly jumping into the HF datasets format

In [ ]:
from datasets import Dataset
from tqdm import tqdm

def json_to_dataset(data):
    datalist = []
    for title in tqdm(data):
        for paragraph in title["paragraphs"]:
            for qa in paragraph["qas"]:
                if len(qa["answers"]) == 0: # bunları dahil edip de kurgulanabilir aslında
                    continue
                example = {'id': qa['id'], 'title': title["title"], 'context': paragraph['context'], 'question': qa['question'], 'answers': qa['answers'][0]}
                datalist.append(example)
    
    return Dataset.from_list(datalist)

In [ ]:
squad_tr = json_to_dataset(qa_data["data"][:1]) # I am limiting the number of titles to 10 for faster computations

100%|██████████| 10/10 [00:00<00:00, 1167.97it/s]

In [ ]:
squad_tr

Split the dataset's `train` split into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [ ]:
squad_tr = squad_tr.train_test_split(test_size=0.2)

In [ ]:
squad_tr

In [ ]:
squad_tr["train"][0]

There are several important fields here:

- `answers`: the starting location of the answer token and the answer text.
- `context`: background information from which the model needs to extract the answer.
- `question`: the question a model should answer.


#### Preprocesing

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/distilbert-base-turkish-cased")

There are a few preprocessing steps particular to question answering tasks you should be aware of:

1. Some examples in a dataset may have a very long `context` that exceeds the maximum input length of the model. To deal with longer sequences, truncate only the `context` by setting `truncation="only_second"`.
2. Next, map the start and end positions of the answer to the original `context` by setting
   `return_offset_mapping=True`.
3. With the mapping in hand, now you can find the start and end tokens of the answer. Use the [sequence_ids](https://huggingface.co/docs/tokenizers/main/en/api/encoding#tokenizers.Encoding.sequence_ids) method to
   find which part of the offset corresponds to the `question` and which corresponds to the `context`.

Here is how you can create a function to truncate and map the start and end tokens of the `answer` to the `context`:

I recommend checking the videos [here](https://huggingface.co/docs/transformers/tasks/question_answering) for grasping the data format for extractive QA, I based most of this section of notebook from that tutorial

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
#tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)
tokenized_squad_tr = squad_tr.map(preprocess_function, batched=True, remove_columns=squad_tr["train"].column_names)

Map:  37%|███▋      | 1000/2696 [00:00<00:00, 2669.54 examples/s]

Map:  74%|███████▍  | 2000/2696 [00:00<00:00, 2700.55 examples/s]

Map: 100%|██████████| 2696/2696 [00:00<00:00, 2711.28 examples/s]

Map: 100%|██████████| 2696/2696 [00:01<00:00, 2675.30 examples/s]

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

Map: 100%|██████████| 675/675 [00:00<00:00, 2493.08 examples/s]

Map: 100%|██████████| 675/675 [00:00<00:00, 2415.62 examples/s]

Now create a batch of examples using [DefaultDataCollator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DefaultDataCollator). Unlike other data collators in 🤗 Transformers, the [DefaultDataCollator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DefaultDataCollator) does not apply any additional preprocessing such as padding.

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

#### Training

In [ ]:
from transformers import AutoModel, AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("dbmdz/distilbert-base-turkish-cased", device_map="cpu")

As a side note, let's see what huggingface mean by "model for question answering" can you spot the difference between when we read the same model as a base model

In [ ]:
model

In [ ]:
base_model = AutoModel.from_pretrained("dbmdz/distilbert-base-turkish-cased")

In [ ]:
base_model

At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model).
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, and data collator.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [ ]:
training_args = TrainingArguments(
    output_dir="test-squad-tr",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_tr["train"],
    eval_dataset=tokenized_squad_tr["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


Let's write a function to peep into what our data looks like at this stage

In [ ]:
def input_data_viewer(data):
    tokens = data["input_ids"]
    padding_start = tokens.index(tokenizer.pad_token_id)
    tokens = tokens[:padding_start]

    #get the answer within
    start = data["start_positions"]
    end = data["end_positions"]

    for idx, token in enumerate(tokens):
        if idx == start:
            print("<<<", end=" ")
        print(tokenizer.decode(token), end=" ")
        if idx == end:
            print(">>>", end=" ")
        

In [ ]:
input_data_viewer(tokenized_squad_tr["train"][2])

In [ ]:
tokenized_squad_tr

In [ ]:
trainer.train()

#### Inference

In [ ]:
question = "SQuAD veriseti ne zaman yayınlandı?"
context = "The Stanford Question Answering Dataset yani SQuAD veriseti 2016 yılında akademik bir kıyaslama veriseti olarak yayınlandı ancak içerdiği basit örnekler eleştirilere sebep oldu"

Tokenize the text and return PyTorch tensors:

In [ ]:
inputs = tokenizer(question, context, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [ ]:
import torch

with torch.no_grad():
    outputs = model(**inputs)

Get the highest probability from the model output for the start and end positions:

In [ ]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

Decode the predicted tokens to get the answer:

In [ ]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

## Sequence Classification

The first example of this notebook was about classification. Sentiment Analysis is one of the most popular sequence classification tasks. Do you think we can formulate a question answering problem as a sequence classification task???

### Sentiment analysis

[winvoker/turkish-sentiment-analysis-dataset](https://huggingface.co/datasets/winvoker/turkish-sentiment-analysis-dataset)

Checking the sequence classification class of bert models will give us an idea about how this problem that we tried to solve with ancient methods, can be solved with language models

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
sc_model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased")

In [ ]:
sc_model

## Language Modeling

### Encoder Models

Modern encoder models take a natural language input and return a contextualised representation of the input.
(still) The most popular and influencial encoder model is BERT.

In [ ]:
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")

Let's see what happens to our text input when passed through an encoder model

In [ ]:
%%time
s = "Bir zamanlar BERTten büyük dil model diye bahsedilirdi..."
inputs = tokenizer(s, return_tensors="pt")
outputs = model(**inputs)

inputs are familiar at this point

In [ ]:
inputs

Let's see what the outputs have to offer

In [ ]:
outputs.__dict__.keys()

Let's dive into what are those and what use they have

In [ ]:
outputs.last_hidden_state.shape

Last hidden state of BERT is shaped like [batch_size, input_token_size, embedding_size] so it generates an embedding vector for each token, which we have utilized for token classification tasks before

In [ ]:
outputs.pooler_output.shape

Pooler output is (although implementations may differ between bert variants) the CLS token embedding went through a linear layer and tanh activation. This is mostly used for sentence embeddings.

In [ ]:
outputs.pooler_output

### Exercise?? if you want so

**This is basically a 768 dimentional feature vector. You can use this for the very first problem in this notebook and see how it compares!**

### Encoder - Decoder Models

Encoder - Decoder Models are mostly used for sequence-to-sequence NLP problems. Such as translation, summarization, generative question answering and so on.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModelForSeq2SeqLM.from_pretrained("ahmetbagci/bert2bert-turkish-paraphrase-generation")

t5tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
t5model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")


/Users/ygzipekci/Library/Application Support/Satyrn/venvs/test_098f6bcd/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "dbmdz/bert-base-turkish-cased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.49

In [2]:
t5model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
text = "beni benden alırsan seni sana bırakmam"
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

### Decoder Models

Decoder Models are all the fuzz since chatgpt. Let's look into their workings

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

We are going to look into instruction tuning.

In [ ]:
from datasets import load_dataset

ds = load_dataset("BrewInteractive/alpaca-tr")

We know that decoder only models are autoregressive next-token predictors. Their task is also called "document completion" because the continue writing whatever the input document was.  
But how come models that just make more of the input receive dialog capabilities?

In [ ]:
tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
def form_prompts(examples):
    prompts = {}
    if examples["input"]:
        messages = [
            {"role": "user", "content": examples["instruction"]},
            {"role": "context", "content": examples["input"]},
            {"role": "assistant", "content": examples["output"]}
        ]
    else:
        messages = [
            {"role": "user", "content": examples["instruction"]},
            {"role": "assistant", "content": examples["output"]}
        ]
    prompts["prompt"] = tokenizer.apply_chat_template(messages, tokenize=False)
    prompts["input_ids"] = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True)
    return {"input_ids": prompts["input_ids"]}

In [ ]:
ds = ds.map(batched_form_prompts, remove_columns=ds["train"].column_names, batched=True)

So yes it is still document completion but the document looks in a very specific format

In [ ]:
ds = ds["train"].train_test_split(test_size=0.2)

In [ ]:
ds

In [ ]:
chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

print(tokenizer.apply_chat_template(chat, tokenize=False))


Every dialog with any instruction model is parsed into a single string at the background

**Extras** What is lora how does it work why does it work?

In [ ]:
model

In [ ]:
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

target_modules = ["c_attn"]
config = LoraConfig(
    r=1,
    lora_alpha=16, 
    target_modules=target_modules, 
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM"
)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = prepare_model_for_kbit_training(model)
lora_model = get_peft_model(model, config)

In [ ]:
lora_model.print_trainable_parameters()

In [ ]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="gpt2_alpaca_tr",
    eval_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01,
    use_cpu=True
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=ds["train"], # datanın neye benzemesi gerekiyo bi bak
    eval_dataset=ds["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [ ]:
ds["train"][0]

In [ ]:
lora_model.device

In [ ]:
trainer.train()

### Inferance

In [ ]:
prompt = "Somatic hypermutation allows the immune system to"

inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)